In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import read_excel
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
file_location = 'input files/definition.xlsx'
df = read_excel(file_location, usecols='B,C,D')
print(df.head())
definitions = []
for i in range(len(df)):
    definitions.append(tuple([df['R'][i],df['G'][i],df['B'][i]]))

# PORTFIX START

In [ ]:
# Fix position, direction, write to port locator file, write to port csv.
### Map dims
maxpixelx = 16384
maxpixely = 6144
### Load port locators
with open("input files/port_locators.txt","r") as file:
    contents = file.read().splitlines()
prov = {}
for line in range(8, len(contents)-4, 6):
    ids = int(contents[line].split("=")[1])
    prov[ids] = contents[line+1].split('{ ')[1].split(' }')[0].split(' ')
    prov[ids] = [int(maxpixely-float(prov[ids][2])),int(float(prov[ids][0]))]
### Which ids correspond to sea provs
seaprovs = list(range(4883,5641+1))
### Load province map
provMap = np.array(Image.open(r"input files/provinces.png","r").convert('RGB'))

In [ ]:
def requirement(loc, provMap, definitions, seaprovs, maxpixely, maxpixelx):
    ylims = [max(0,loc[0]-1), min(loc[0]+1,maxpixely-1)]
    xlims = [max(0,loc[1]-1), min(loc[1]+1,maxpixelx-1)]

    loct = [[ylims[0], loc[1]], [ylims[1], loc[1]], [loc[0], xlims[0]], [loc[0], xlims[1]]]
    for i in range(4):
        color = tuple(provMap[tuple(loct[i])])
        for ids2, value in enumerate(definitions):
            if value==color:
                break
        if ids2 in seaprovs: return True, ids2

    return False, None

# Fix location
def inner_loop(ids, prov, provMap, definitions, seaprovs, maxpixely, maxpixelx):
    loc = prov[ids]
    provcolor = definitions[ids]
    print(provcolor)
    # Floodfill through the provinces until we find the right spot. Province pixel next to sea
    swarm = [loc]
    while len(swarm)!=0:
        yx = swarm[0]

        output = requirement(yx, provMap, definitions, seaprovs, maxpixely, maxpixelx)
        print(swarm)
        if output[0] is True: break

        # Keep searching
        ylims = [max(0,yx[0]-1), min(yx[0]+1,maxpixely-1)]
        xlims = [max(0,yx[1]-1), min(yx[1]+1,maxpixelx-1)]

        loct = [[ylims[0], yx[1]], [ylims[1], yx[1]], [yx[0], xlims[0]], [yx[0], xlims[1]]]
        print(loct)
        for i in range(4):
            color = tuple(provMap[tuple(loct[i])])
            if color==provcolor:
                swarm.append(loct[i])
                provMap[tuple(loct[i])] = (0,0,0)
        print(swarm)

        swarm.pop(0)
    return yx, output[1]

#result = Parallel(n_jobs=-2)(delayed(inner_loop)(ids, provMap, definitions, seaprovs, maxpixely, maxpixelx) for ids in tqdm(prov))

for ids in prov:
    result = inner_loop(ids, prov, provMap, definitions, seaprovs, maxpixely, maxpixelx)

In [ ]:
prov

In [ ]:
result

# END

In [ ]:
seaprovs = list(range(14317, 15075+1))
bprov = (255,255,0)

Image.MAX_IMAGE_PIXELS = None
provimage = Image.open(r'input files/provinces.png')
provimage.convert('RGB')

data = np.array(provimage)   # "data" is a height x width x 4 numpy array
r, g, b = data.T # Temporarily unpack the bands for readability

interest_areas = (r == bprov[0]) & (g == bprov[1]) & (b == bprov[2])

for ids in seaprovs:
    color = definitions[ids]
    interest_areas = (r == color[0]) & (g == color[1]) & (b == color[2]) | interest_areas

data[..., :][interest_areas.T] = bprov # Transpose back needed

provimage = Image.fromarray(data)
provimage.save('yellowseas.png')
provimage.close()

In [ ]:
#From default.map we check the provinces that are sea zones, making a list with them
seaprovs = list(range(14317, 15075+1))
bprov = (255,255,0)
fillcolor = (250,250,250)
mapwidth = 16384
mapheight = 6144

#Load province automatically assigned port locators
f = open("input files/port_locators.txt","r")
contents = f.read().splitlines()
f.close()
coastalprovs = {}
for line in range(8, len(contents)-4, 6):
    ids = int(contents[line].split("=")[1])
    temp = contents[line+1].split(" ")[1:-1]
    coastalprovs[ids] = {
        'x': int(float(temp[0])),
        'y': mapheight-int(float(temp[2])),
        'portx': None,
        'porty': None,
        'cityx': None,
        'cityy': None,
        'assigned': None
    }

#Load province map, iterate through every province with a port locator [coastalprovs], iterate through the neighborhood of the locator to find the nearest sea province [seaprovs]
provimage = Image.open(r'input files/provinces.png')
#Load province map with yellow seas
yellowimg = Image.open('yellowseas.png')

# def seafinder(found, pixelcolor):
#     for ids in seaprovs:
#         if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]):
#             found = True
#             break
#     return ids, found

for p in coastalprovs:
    x0, y0 = coastalprovs[p]['x'], coastalprovs[p]['y']
    pixelcolor = provimage.getpixel((x0,y0))

    #Some provinces with weird shapes have their centers outside of the province itself, we look for the nearest
    if 3 != sum([1 for a, b in zip(pixelcolor, definitions[p]) if a == b]):
        radius = 0
        while True:
            radius+=1
            x = x0
            y = max(y0 - radius,0)
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[p]) if a == b]): break
            x = max(x0 - radius,0)
            y = y0
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[p]) if a == b]): break
            x = x0
            y = min(y0 + radius,mapheight-1)
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[p]) if a == b]): break
            x = min(x0 + radius,mapwidth-1)
            y = y0
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[p]) if a == b]): break

            if radius>100: print(p)

        x0, y0 = x, y
        coastalprovs[p]['x'], coastalprovs[p]['y'] = x, y

    #Now that the original pixel belongs to the correct province, we floodfill until finding a yellow pixel, detect the sea and continue loop
    yellowimg.putpixel((x0,y0),fillcolor)
    pixel_list = [(x0,y0)]
    i = 0
    while i<len(pixel_list):
        #Flood fill the province with the fill color
        templeft = (pixel_list[i][0]-1,pixel_list[i][1])
        if templeft[0]>-1:
            tempcolor = yellowimg.getpixel(templeft)
            if 3 == sum([1 for a, b in zip(tempcolor, bprov) if a == b]): 
                x, y = templeft
                break
            if 3 == sum([1 for a, b in zip(tempcolor, pixelcolor) if a == b]):
                pixel_list.append(templeft)
                yellowimg.putpixel(templeft,fillcolor)
        tempright = (pixel_list[i][0]+1,pixel_list[i][1])
        if tempright[0]<mapwidth:
            tempcolor = yellowimg.getpixel(tempright)
            if 3 == sum([1 for a, b in zip(tempcolor, bprov) if a == b]): 
                x, y = tempright
                break
            if 3 == sum([1 for a, b in zip(tempcolor, pixelcolor) if a == b]):
                pixel_list.append(tempright)
                yellowimg.putpixel(tempright,fillcolor)
        tempdown = (pixel_list[i][0],pixel_list[i][1]+1)
        if tempdown[1]<mapheight:
            tempcolor = yellowimg.getpixel(tempdown)
            if 3 == sum([1 for a, b in zip(tempcolor, bprov) if a == b]): 
                x, y = tempdown
                break
            if 3 == sum([1 for a, b in zip(tempcolor, pixelcolor) if a == b]):
                pixel_list.append(tempdown)
                yellowimg.putpixel(tempdown,fillcolor)
        temptop = (pixel_list[i][0],pixel_list[i][1]-1)
        if temptop[1]>-1:
            tempcolor = yellowimg.getpixel(temptop)
            if 3 == sum([1 for a, b in zip(tempcolor, bprov) if a == b]): 
                x, y = temptop
                break
            if 3 == sum([1 for a, b in zip(tempcolor, pixelcolor) if a == b]):
                pixel_list.append(temptop)
                yellowimg.putpixel(temptop,fillcolor)
        i+=1

    tempcolor = provimage.getpixel((x,y))
    for ids in seaprovs:
        if 3 == sum([1 for a, b in zip(tempcolor, definitions[ids]) if a == b]):
            coastalprovs[p]['assigned'] = ids
            break

    coastalprovs[p]['portx'], coastalprovs[p]['porty'] = pixel_list[i][0], pixel_list[i][1]
    coastalprovs[p]['cityx'], coastalprovs[p]['cityy'] = int((coastalprovs[p]['portx']+x0)/2), int((coastalprovs[p]['porty']+y0)/2)
    
provimage.close()
yellowimg.close()

In [ ]:
#Writing the excel port file
dfports = {
    'LandProvince': [],
    'SeaZone': [],
    'x': [],
    'y;': [],
}
for p in coastalprovs:
    dfports['LandProvince'].append(p)
    dfports['SeaZone'].append(coastalprovs[p]['assigned'])
    dfports['x'].append(coastalprovs[p]['portx'])
    dfports['y;'].append(str(mapheight-coastalprovs[p]['porty'])+';')

dfports['LandProvince'].append(-1)
dfports['SeaZone'].append(-1)
dfports['x'].append(-1)
dfports['y;'].append(str(-1)+';')

dfports = pd.DataFrame(data=dfports)
dfports.to_excel('output files/ports.xlsx', index=False)
print(dfports.head())

In [ ]:
#Writing the proper port locators
with open('output files/port_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="port"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer=""\n\tinstances={')
    for p in coastalprovs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str(coastalprovs[p]['portx'])+' 0.0 '+str(mapheight-coastalprovs[p]['porty'])+' }')
        f.write('\n\t\t\trotation={ -0.0 -0.0 -0.0 1.0 }')
        f.write('\n\t\t\tscale={ 1.0 1.0 1.0 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')

#Seagull locators
with open('output files/animal_locators.txt','w') as f:
    f.write('object={\n\tname="seagulls"\n\tclamp_to_water_level=yes\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="monument_layer"\n\tentity="seagull_port_entity"')
    f.write('\n\tcount='+str(len(coastalprovs))+'\n\ttransform="')
    for p in coastalprovs:
        f.write('\n'+str(coastalprovs[p]['portx'])+' 0.0 '+str(mapheight-coastalprovs[p]['porty']))
        f.write(' -0.0 '+str(round(np.random.uniform(-1,0),4))+' -0.0 '+str(round(np.random.uniform(-1,1),4)))
        f.write(' 1.0 1.0 1.0')
    f.write('\n"}')

In [ ]:
#Load land province automatically assigned centers
provimage = Image.open(r'input files/provinces.png')
f = open("input files/fort_locators.txt","r")
contents = f.read().splitlines()
f.close()

landprovs = {}
for line in range(8, len(contents)-4, 6):
    ids = int(contents[line].split("=")[1])
    if ids in coastalprovs:
        landprovs[ids] = {
            'x': coastalprovs[ids]['cityx'],
            'y': coastalprovs[ids]['cityy'],
            'cityx': coastalprovs[ids]['x'],
            'cityy': coastalprovs[ids]['y']
        }
        continue

    temp = contents[line+1].split(" ")[1:-1]
    landprovs[ids] = {
        'x': None,
        'y': None,
        'cityx': int(float(temp[0])),
        'cityy': mapheight-int(float(temp[2]))
    }

    x0, y0 = landprovs[ids]['cityx'], landprovs[ids]['cityy']
    pixelcolor = provimage.getpixel((x0,y0))
    if 3 != sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]):
        radius = 0
        while True:
            radius+=1
            x = x0
            y = max(y0 - radius,0)
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
            x = max(x0 - radius,0)
            y = y0
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
            x = x0
            y = min(y0 + radius,mapheight-1)
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
            x = min(x0 + radius,mapwidth-1)
            y = y0
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break

            if radius>100: print(ids)

        x0, y0 = x, y
        landprovs[ids]['cityx'], landprovs[ids]['cityy'] = x, y

    while True:
        x = x0+np.random.randint(-20,21)
        y = y0+np.random.randint(-20,21)
        pixelcolor = provimage.getpixel((x,y))
        if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
    landprovs[ids]['x'], landprovs[ids]['y'] = x, y

provimage.close()

In [ ]:
#Writing the proper fort & city locators
with open('output files/fort_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="fort"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="forts_layer"\n\tinstances={')
    for p in landprovs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str(landprovs[p]['x'])+' 0.0 '+str(mapheight-landprovs[p]['y'])+' }')
        f.write('\n\t\t\trotation={ -0.0 '+str(round(np.random.uniform(-1,0),4))+' -0.0 '+str(round(np.random.uniform(-1,1),4))+' }')
        f.write('\n\t\t\tscale={ 0.7 0.7 0.7 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')
with open('output files/city_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="city"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="cities_layer"\n\tinstances={')
    for p in landprovs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str(landprovs[p]['cityx'])+' 0.0 '+str(mapheight-landprovs[p]['cityy'])+' }')
        f.write('\n\t\t\trotation={ -0.0 '+str(round(np.random.uniform(-1,0),4))+' -0.0 '+str(round(np.random.uniform(-1,1),4))+' }')
        f.write('\n\t\t\tscale={ 1.2 1.0 1.2 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')
with open('output files/great_work_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="great_work"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="monument_layer"\n\tinstances={')
    for p in landprovs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str((landprovs[p]['cityx']+landprovs[p]['x'])/2)+' 0.0 '+str(mapheight-(landprovs[p]['cityy']+landprovs[p]['y'])/2)+' }')
        f.write('\n\t\t\trotation={ -0.0 '+str(round(np.random.uniform(-1,0),4))+' -0.0 '+str(round(np.random.uniform(-1,1),4))+' }')
        f.write('\n\t\t\tscale={ 1.1 1.0 1.1 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')

In [ ]:
#Load province automatically assigned centers
provimage = Image.open(r'input files/provinces.png')
f = open("input files/combat_locators.txt","r")
contents = f.read().splitlines()
f.close()

provs = {}
for line in range(8, len(contents)-4, 6):
    ids = int(contents[line].split("=")[1])
    if ids in landprovs:
        provs[ids] = {
            'x': landprovs[ids]['cityx'],
            'y': landprovs[ids]['cityy']
        }
        continue

    temp = contents[line+1].split(" ")[1:-1]
    provs[ids] = {
        'x': int(float(temp[0])),
        'y': mapheight-int(float(temp[2]))
    }

    x0, y0 = provs[ids]['x'], provs[ids]['y']
    if ids == 0: continue
    pixelcolor = provimage.getpixel((x0,y0))
    if 3 != sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]):
        print(str(ids)+' has happened')
        radius = 0
        while True:
            radius+=1
            x = x0
            y = max(y0 - radius,0)
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
            x = max(x0 - radius,0)
            y = y0
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
            x = x0
            y = min(y0 + radius,mapheight-1)
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break
            x = min(x0 + radius,mapwidth-1)
            y = y0
            pixelcolor = provimage.getpixel((x,y))
            if 3 == sum([1 for a, b in zip(pixelcolor, definitions[ids]) if a == b]): break

            if radius>100: print(ids)

        provs[ids]['x'], provs[ids]['y'] = x, y


provimage.close()

In [ ]:
#Writing the proper combat and unit stack locators
with open('output files/combat_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="combat"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="unit_layer"\n\tinstances={')
    for p in provs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str(provs[p]['x'])+' 0.0 '+str(mapheight-provs[p]['y'])+' }')
        f.write('\n\t\t\trotation={ -0.0 -0.0 -0.0 1.0 }')
        f.write('\n\t\t\tscale={ 1.0 1.0 1.0 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')
with open('output files/unit_stack_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="unit_stack"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="unit_layer"\n\tinstances={')
    for p in provs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str(provs[p]['x'])+' 0.0 '+str(mapheight-provs[p]['y'])+' }')
        f.write('\n\t\t\trotation={ -0.0 -0.0 -0.0 1.0 }')
        f.write('\n\t\t\tscale={ 1.0 1.0 1.0 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')
with open('output files/vfx_locators.txt','w') as f:
    f.write('game_object_locator={\n\tname="vfx"\n\tclamp_to_water_level=no\n\trender_under_water=no\n\tgenerated_content=no\n\tlayer="vfx_layer"\n\tinstances={')
    for p in provs:
        f.write('\n\t\t{\n\t\t\tid='+str(p))
        f.write('\n\t\t\tposition={ '+str(provs[p]['x'])+' 0.0 '+str(mapheight-provs[p]['y'])+' }')
        f.write('\n\t\t\trotation={ -0.0 -0.0 -0.0 1.0 }')
        f.write('\n\t\t\tscale={ 1.0 1.0 1.0 }')
        f.write('\n\t\t}')
    f.write('\n\t}\n}')